# Example of Using ComBlock with Pynq

For convenience, the *comblock.py* Python script is established which contains useful constants for interacting with the ComBlock.

In [1]:
from pynq import Overlay
from pynq import MMIO
import comblock as cbc

In [ ]:
# Load the overlay (bitstream) onto the FPGA. In the Overlays folder, you can choose if you are using a Z1 or Z2 Pynq board.
base = Overlay("pynq_z1_comblock.bit")

The information from the **comblock_0** block is read to verify everything that is obtained. Since the object is mapped to AXI Lite, it is noted that the AXI Full address is omitted.

In [6]:
base.ip_dict['comblock_0']

{'type': 'www.ictp.it:user:comblock:2.0',
 'mem_id': 'AXIF',
 'memtype': 'REGISTER',
 'gpio': {},
 'interrupts': {},
 'parameters': {'REGS_IN_ENA': 'true',
  'REGS_IN_DWIDTH': '32',
  'REGS_IN_DEPTH': '4',
  'REGS_OUT_ENA': 'true',
  'REGS_OUT_DWIDTH': '32',
  'REGS_OUT_DEPTH': '4',
  'DRAM_IO_ENA': 'true',
  'DRAM_IO_DWIDTH': '16',
  'DRAM_IO_AWIDTH': '16',
  'DRAM_IO_DEPTH': '0',
  'FIFO_IN_ENA': 'true',
  'FIFO_IN_DWIDTH': '16',
  'FIFO_IN_DEPTH': '1024',
  'FIFO_IN_AFOFFSET': '1',
  'FIFO_IN_AEOFFSET': '1',
  'FIFO_OUT_ENA': 'false',
  'FIFO_OUT_DWIDTH': '16',
  'FIFO_OUT_DEPTH': '1024',
  'FIFO_OUT_AFOFFSET': '1',
  'FIFO_OUT_AEOFFSET': '1',
  'C_AXIF_ID_WIDTH': '12',
  'C_AXIF_AWUSER_WIDTH': '0',
  'C_AXIF_ARUSER_WIDTH': '0',
  'C_AXIF_WUSER_WIDTH': '0',
  'C_AXIF_RUSER_WIDTH': '0',
  'C_AXIF_BUSER_WIDTH': '0',
  'Component_Name': 'pynqCB_comblock_0_0',
  'EDK_IPTYPE': 'PERIPHERAL',
  'C_AXIL_BASEADDR': '0x43C00000',
  'C_AXIL_HIGHADDR': '0x43C0FFFF',
  'C_AXIF_BASEADDR': '0x43C4

In [7]:
# The object is created based on the comblock_0 IP
cb = base.comblock_0

## Interacting with basic elements of ComBlock (I/O Reginsters, RAM, FIFO)

### Read and write registers

In [32]:
#write and read registers
#cd.write(register, data)
cb.write(cbc.OREG0,20)
#cd.read(register)
cb.read(cbc.IREG0)

20

In [ ]:
#write and read registers with xor (example design)
#cd.write(register, data)
cb.write(cbc.OREG1,20)
#cd.read(register)
cb.read(cbc.IREG1)

In [33]:
#read push buttons and slide switches, all the buttons btn and sw are as follows: MSB(bnt3, bnt2, bnt1, btn0, sw1, sw0)LSB
cb.read(cbc.IREG2)

0

In [34]:
#write leds (1 led0, 2 led1, 4 led2, 8 led3)
cb.write(cbc.OREG2,0)

In [ ]:
#write leds rgb  (1 blue led4, 2 green led4, 4, red led4, 8 blue led5, 16 green led5, 32 red led5)
cb.write(cbc.OREG3,32)

### Read FIFO

In [35]:
# From the ComBlock IP, it is possible to obtain its parameters and define them as variables.
FIFO_DEPTH = int(base.ip_dict['comblock_0']['parameters']['FIFO_OUT_DEPTH'])
print(FIFO_DEPTH)
valor = []
for i in range(10):
    val = cb.read(cbc.IFIFO_VALUE)
    valor.append(val)
    
print(valor)

1024
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


### Read RAM

In [36]:
# The width of the addresses and data is defined.
RAM_ADDR_WIDTH = int(base.ip_dict['comblock_0']['parameters']['DRAM_IO_AWIDTH'])
RAM_DATA_WIDTH = int(base.ip_dict['comblock_0']['parameters']['DRAM_IO_DWIDTH'])
# From the block design in Vivado, the base address of AXI Full is obtained, which is where the ComBlock manages the RAM.
RAM_BASE_ADDR = int(base.ip_dict['comblock_0']['parameters']['C_AXIF_BASEADDR'],16)
# The object is created based on the MMIO class, which requires the base address and its width.
# To read in multiples of 4, you must multiply by 4
cbRam = MMIO(RAM_BASE_ADDR, RAM_ADDR_WIDTH*4)

# Finally, a loop is performed for each memory address that needs to be accessed, ensuring they are multiples of 4. Additionally, the function is enhanced to specify the data width, since reading bytes involves dividing the MMIO limit by 4.
for i in range(RAM_DATA_WIDTH):
    print(cbRam.read(i*4,RAM_DATA_WIDTH//4))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
